# Anticipez les besoins en consommation de bâtiments - *Notebook prediction*

## Mission

Vous travaillez pour la ville de Seattle. Pour atteindre son objectif de ville neutre en émissions de carbone en 2050, votre équipe s’intéresse de près à la consommation et aux émissions des bâtiments non destinés à l’habitation.

Des relevés minutieux ont été effectués par les agents de la ville en 2016. Cependant, ces relevés sont coûteux à obtenir, et à partir de ceux déjà réalisés, vous voulez tenter de prédire les émissions de CO2 et la consommation totale d’énergie de bâtiments non destinés à l’habitation pour lesquels elles n’ont pas encore été mesurées.

Votre prédiction se basera sur les données structurelles des bâtiments (taille et usage des bâtiments, date de construction, situation géographique, ...)

Vous cherchez également à évaluer l’intérêt de l’ENERGY STAR Score pour la prédiction d’émissions, qui est fastidieux à calculer avec l’approche utilisée actuellement par votre équipe. Vous l'intégrerez dans la modélisation et jugerez de son intérêt.

Vous sortez tout juste d’une réunion de brief avec votre équipe. Voici un récapitulatif de votre mission :


1) Réaliser une courte analyse exploratoire.
2) Tester différents modèles de prédiction afin de répondre au mieux à la problématique.

Fais bien attention au traitement des différentes variables, à la fois pour trouver de nouvelles informations (peut-on déduire des choses intéressantes d’une simple adresse ?) et optimiser les performances en appliquant des transformations simples aux variables (normalisation, passage au log, etc.).

Mets en place une évaluation rigoureuse des performances, et optimise les hyperparamètres et le choix d’algorithmes de ML à l’aide d’une validation croisée. Tu testeras au minimum 4 algorithmes de famille différente (par exemple : ElasticNet, SVM, GradientBoosting, RandomForest).

In [483]:
import numpy as np

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import FuncFormatter
import scipy
from scipy import stats
import scipy.stats as st

import statsmodels
import statsmodels.api as sm
import missingno as msno

import sklearn
from sklearn.experimental import enable_iterative_imputer  # Nécessaire pour activer IterativeImputer
from sklearn.impute import IterativeImputer

from sklearn.impute import KNNImputer
# Encodage des variables catégorielles avant d'utiliser KNNImputer
from category_encoders.ordinal import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# pour le centrage et la réduction
from sklearn.preprocessing import StandardScaler
# pour l'ACP
from sklearn.decomposition import PCA

from sklearn import model_selection
from sklearn.model_selection import GridSearchCV

from sklearn import metrics
from sklearn.metrics import roc_curve, auc, confusion_matrix, mean_squared_error, make_scorer, r2_score, mean_absolute_error

from sklearn import dummy
from sklearn.dummy import DummyClassifier
from sklearn.dummy import DummyRegressor

from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LogisticRegression

from sklearn.svm import LinearSVC
from sklearn.svm import SVR

from sklearn import kernel_ridge

from sklearn import neighbors
from sklearn.neighbors import KNeighborsClassifier

import tensorflow
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from xgboost import XGBRegressor

import timeit
import warnings

print("numpy version", np.__version__)
print("pandas version", pd.__version__)
print("matplotlib version", matplotlib.__version__)
print("seaborn version", sns.__version__)
print("scipy version", scipy.__version__)
print("statsmodels version", statsmodels.__version__)
print("missingno version", msno.__version__)

print("sklearn version", sklearn.__version__)
print("tensorflow version", tensorflow.__version__)

pd.options.display.max_rows = 200
pd.options.display.max_columns = 100

numpy version 1.26.4
pandas version 2.1.4
matplotlib version 3.8.0
seaborn version 0.13.2
scipy version 1.11.4
statsmodels version 0.14.0
missingno version 0.5.2
sklearn version 1.2.2
tensorflow version 2.18.0


## 1 - Développement et simulation du premier modèle (cible = SiteEnergyUseWN(kBtu))

In [123]:
# Charger le fichier de données
data_fe1 = pd.read_csv("C:/Users/admin/Documents/Projets/Projet_4/data_projet/cleaned/2016_Building_Energy_Benchmarking_fe1.csv", sep=',', low_memory=False)
data_fe1.head()

,NumberofBuildings,NumberofFloors,PropertyGFATotal,SiteEnergyUseWN(kBtu),TotalGHGEmissions,PrimaryPropertyType_Distribution Center,PrimaryPropertyType_Hospital,PrimaryPropertyType_Hotel,PrimaryPropertyType_K-12 School,PrimaryPropertyType_Laboratory,PrimaryPropertyType_Large Office,PrimaryPropertyType_Low-Rise Multifamily,PrimaryPropertyType_Medical Office,PrimaryPropertyType_Mixed Use Property,PrimaryPropertyType_Office,PrimaryPropertyType_Other,PrimaryPropertyType_Refrigerated Warehouse,PrimaryPropertyType_Residence Hall,PrimaryPropertyType_Restaurant,PrimaryPropertyType_Retail Store,PrimaryPropertyType_Self-Storage Facility,PrimaryPropertyType_Senior Care Community,PrimaryPropertyType_Small- and Mid-Sized Office,PrimaryPropertyType_Supermarket / Grocery Store,PrimaryPropertyType_University,PrimaryPropertyType_Warehouse,PrimaryPropertyType_Worship Facility,Neighborhood_BALLARD,Neighborhood_CENTRAL,Neighborhood_DELRIDGE,Neighborhood_DELRIDGE NEIGHBORHOODS,Neighborhood_DOWNTOWN,Neighborhood_EAST,Neighborhood_GREATER DUWAMISH,Neighborhood_LAKE UNION,Neighborhood_MAGNOLIA / QUEEN ANNE,Neighborhood_NORTH,Neighborhood_NORTHEAST,Neighborhood_NORTHWEST,Neighborhood_SOUTHEAST,Neighborhood_SOUTHWEST,"YearBuilt_Bin_(1899.885, 1911.5]","YearBuilt_Bin_(1911.5, 1923.0]","YearBuilt_Bin_(1923.0, 1934.5]","YearBuilt_Bin_(1934.5, 1946.0]","YearBuilt_Bin_(1946.0, 1957.5]","YearBuilt_Bin_(1957.5, 1969.0]","YearBuilt_Bin_(1969.0, 1980.5]","YearBuilt_Bin_(1980.5, 1992.0]","YearBuilt_Bin_(1992.0, 2003.5]","YearBuilt_Bin_(2003.5, 2015.0]",electricity_percent,gaz_percent,steam_percent
0,1.000,12,88434.000,7456910.000,249.980,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,54.610,17.660,27.730
1,1.000,11,103566.000,8664479.000,295.860,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,38.660,61.340,0.000
2,1.000,10,61320.000,6946800.500,286.430,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,40.750,26.660,32.590
3,1.000,18,175580.000,14656503.000,505.010,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,37.880,62.120,0.000
4,1.000,2,97288.000,12581712.000,301.810,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,60.990,39.010,0.000


In [124]:
data_fe1.shape

(1444, 54)

### 1.1 - Sélectionner les features et la cible :

In [126]:
y_fe1_conso = data_fe1['SiteEnergyUseWN(kBtu)']
X_fe1 = data_fe1.drop('SiteEnergyUseWN(kBtu)', axis=1, inplace=False)
X_fe1.shape

(1444, 53)

In [127]:
y_fe1_conso.shape

(1444,)

In [128]:
y_fe1_emissions = data_fe1['TotalGHGEmissions']
X_fe1 = X_fe1.drop('TotalGHGEmissions', axis=1, inplace=False)
X_fe1.shape

(1444, 52)

In [129]:
y_fe1_emissions.shape

(1444,)

In [479]:
y_fe1_conso.mean()

3770361.464915118

In [477]:
data_fe1.describe()

,NumberofBuildings,NumberofFloors,PropertyGFATotal,SiteEnergyUseWN(kBtu),TotalGHGEmissions,PrimaryPropertyType_Distribution Center,PrimaryPropertyType_Hospital,PrimaryPropertyType_Hotel,PrimaryPropertyType_K-12 School,PrimaryPropertyType_Laboratory,PrimaryPropertyType_Large Office,PrimaryPropertyType_Low-Rise Multifamily,PrimaryPropertyType_Medical Office,PrimaryPropertyType_Mixed Use Property,PrimaryPropertyType_Office,PrimaryPropertyType_Other,PrimaryPropertyType_Refrigerated Warehouse,PrimaryPropertyType_Residence Hall,PrimaryPropertyType_Restaurant,PrimaryPropertyType_Retail Store,PrimaryPropertyType_Self-Storage Facility,PrimaryPropertyType_Senior Care Community,PrimaryPropertyType_Small- and Mid-Sized Office,PrimaryPropertyType_Supermarket / Grocery Store,PrimaryPropertyType_University,PrimaryPropertyType_Warehouse,PrimaryPropertyType_Worship Facility,Neighborhood_BALLARD,Neighborhood_CENTRAL,Neighborhood_DELRIDGE,Neighborhood_DELRIDGE NEIGHBORHOODS,Neighborhood_DOWNTOWN,Neighborhood_EAST,Neighborhood_GREATER DUWAMISH,Neighborhood_LAKE UNION,Neighborhood_MAGNOLIA / QUEEN ANNE,Neighborhood_NORTH,Neighborhood_NORTHEAST,Neighborhood_NORTHWEST,Neighborhood_SOUTHEAST,Neighborhood_SOUTHWEST,"YearBuilt_Bin_(1899.885, 1911.5]","YearBuilt_Bin_(1911.5, 1923.0]","YearBuilt_Bin_(1923.0, 1934.5]","YearBuilt_Bin_(1934.5, 1946.0]","YearBuilt_Bin_(1946.0, 1957.5]","YearBuilt_Bin_(1957.5, 1969.0]","YearBuilt_Bin_(1969.0, 1980.5]","YearBuilt_Bin_(1980.5, 1992.0]","YearBuilt_Bin_(1992.0, 2003.5]","YearBuilt_Bin_(2003.5, 2015.0]",electricity_percent,gaz_percent,steam_percent
count,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000
mean,1.053,3.076,68869.966,3770361.465,78.810,0.035,0.001,0.042,0.085,0.003,0.071,0.001,0.019,0.067,0.002,0.147,0.008,0.014,0.008,0.057,0.019,0.010,0.197,0.026,0.010,0.129,0.046,0.045,0.033,0.030,0.001,0.189,0.071,0.226,0.084,0.090,0.044,0.076,0.054,0.030,0.026,0.105,0.078,0.095,0.033,0.109,0.158,0.115,0.107,0.111,0.089,69.171,29.157,1.672
std,0.396,3.747,68751.053,3595093.942,95.627,0.185,0.026,0.201,0.279,0.053,0.257,0.037,0.138,0.250,0.046,0.354,0.087,0.117,0.091,0.233,0.138,0.101,0.398,0.160,0.101,0.335,0.210,0.207,0.178,0.170,0.026,0.392,0.257,0.419,0.278,0.286,0.206,0.265,0.226,0.170,0.160,0.306,0.269,0.293,0.179,0.311,0.365,0.319,0.310,0.315,0.284,26.608,26.708,8.286
min,1.000,0.000,11285.000,58114.199,0.400,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,1.000,1.000,28062.000,1259691.750,18.627,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,47.797,0.000,0.000
50%,1.000,2.000,43653.000,2343374.125,43.540,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,69.190,26.905,0.000
75%,1.000,4.000,78380.750,5206861.750,95.877,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0

In [130]:
X_fe1.head()

,NumberofBuildings,NumberofFloors,PropertyGFATotal,PrimaryPropertyType_Distribution Center,PrimaryPropertyType_Hospital,PrimaryPropertyType_Hotel,PrimaryPropertyType_K-12 School,PrimaryPropertyType_Laboratory,PrimaryPropertyType_Large Office,PrimaryPropertyType_Low-Rise Multifamily,PrimaryPropertyType_Medical Office,PrimaryPropertyType_Mixed Use Property,PrimaryPropertyType_Office,PrimaryPropertyType_Other,PrimaryPropertyType_Refrigerated Warehouse,PrimaryPropertyType_Residence Hall,PrimaryPropertyType_Restaurant,PrimaryPropertyType_Retail Store,PrimaryPropertyType_Self-Storage Facility,PrimaryPropertyType_Senior Care Community,PrimaryPropertyType_Small- and Mid-Sized Office,PrimaryPropertyType_Supermarket / Grocery Store,PrimaryPropertyType_University,PrimaryPropertyType_Warehouse,PrimaryPropertyType_Worship Facility,Neighborhood_BALLARD,Neighborhood_CENTRAL,Neighborhood_DELRIDGE,Neighborhood_DELRIDGE NEIGHBORHOODS,Neighborhood_DOWNTOWN,Neighborhood_EAST,Neighborhood_GREATER DUWAMISH,Neighborhood_LAKE UNION,Neighborhood_MAGNOLIA / QUEEN ANNE,Neighborhood_NORTH,Neighborhood_NORTHEAST,Neighborhood_NORTHWEST,Neighborhood_SOUTHEAST,Neighborhood_SOUTHWEST,"YearBuilt_Bin_(1899.885, 1911.5]","YearBuilt_Bin_(1911.5, 1923.0]","YearBuilt_Bin_(1923.0, 1934.5]","YearBuilt_Bin_(1934.5, 1946.0]","YearBuilt_Bin_(1946.0, 1957.5]","YearBuilt_Bin_(1957.5, 1969.0]","YearBuilt_Bin_(1969.0, 1980.5]","YearBuilt_Bin_(1980.5, 1992.0]","YearBuilt_Bin_(1992.0, 2003.5]","YearBuilt_Bin_(2003.5, 2015.0]",electricity_percent,gaz_percent,steam_percent
0,1.000,12,88434.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,54.610,17.660,27.730
1,1.000,11,103566.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,38.660,61.340,0.000
2,1.000,10,61320.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,40.750,26.660,32.590
3,1.000,18,175580.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,37.880,62.120,0.000
4,1.000,2,97288.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,60.990,39.010,0.000


### 1.2 - Standardiser les valeurs et créer les jeux d'entraînement / test

In [132]:
X_scale_fe1 = StandardScaler().fit_transform(X_fe1)
#X_scale_fe1 = std_scale.transform(X_fe1)

In [133]:
df = pd.DataFrame(X_scale_fe1)
df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51
count,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000
mean,0.000,0.000,-0.000,-0.000,0.000,-0.000,-0.000,-0.000,0.000,0.000,0.000,0.000,-0.000,-0.000,-0.000,0.000,-0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.000,-0.000,-0.000,-0.000,-0.000,-0.000,-0.000,-0.000,-0.000,0.000,0.000,-0.000,0.000,-0.000,-0.000,0.000,-0.000,-0.000,-0.000,0.000,0.000,-0.000,0.000,0.000,0.000,-0.000
std,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
min,-0.133,-0.821,-0.838,-0.191,-0.026,-0.210,-0.305,-0.053,-0.277,-0.037,-0.141,-0.268,-0.046,-0.415,-0.088,-0.119,-0.092,-0.247,-0.141,-0.102,-0.495,-0.164,-0.102,-0.385,-0.221,-0.217,-0.183,-0.175,-0.026,-0.483,-0.277,-0.541,-0.304,-0.315,-0.215,-0.287,-0.239,-0.175,-0.164,-0.342,-0.291,-0.324,-0.185,-0.349,-0.433,-0.360,-0.347,-0.354,-0.312,-2.601,-1.092,-0.202
25%,-0.133,-0.554,-0.594,-0.191,-0.026,-0.210,-0.305,-0.053,-0.277,-0.037,-0.141,-0.268,-0.046,-0.415,-0.088,-0.119,-0.092,-0.247,-0.141,-0.102,-0.495,-0.164,-0.102,-0.385,-0.221,-0.217,-0.183,-0.175,-0.026,-0.483,-0.277,-0.541,-0.304,-0.315,-0.215,-0.287,-0.239,-0.175,-0.164,-0.342,-0.291,-0.324,-0.185,-0.349,-0.433,-0.360,-0.347,-0.354,-0.312,-0.804,-1.092,-0.202
50%,-0.133,-0.287,-0.367,-0.191,-0.026,-0.210,-0.305,-0.053,-0.277,-0.037,-0.141,-0.268,-0.046,-0.415,-0.088,-0.119,-0.092,-0.247,-0.141,-0.102,-0.495,-0.164,-0.102,-0.385,-0.221,-0.217,-0.183,-0.175,-0.026,-0.483,-0.277,-0.541,-0.304,-0.315,-0.215,-0.287,-0.239,-0.175,-0.164,-0.342,-0.291,-0.324,-0.185,-0.349,-0.433,-0.360,-0.347,-0.354,-0.312,0.001,-0.084,-0.202
75%,-0.133,0.247,0.138,-0.191,-0.026,-0.210,-0.305,-0.053,-0.277,-0.037,-0.141,-0.268,-0.046,-0.415,-0.088,-0.119,-0.092,-0.247,-0.141,-0.102,-0.495,-0.164,-0.102,-0.385,-0.221,-0.217,-0.183,-0.175,-0.026,-0.483,-0.277,-0.541,-0.304,-0.315,-0.215,-0.287,-0.239,-0.175,-0.164,-0.342,-0.291,-0.324,-0.185,-0.349,-0.433,-0.360,-0.347,-0.354,-0.312,1.159,0.818,-0.202
max,12.506,25.612,6.807,5.226,37.987,4.762,3.277,18.974,3.608,26.851,7.111,3.726,21.917,2.411,11.414,8.438,10.924,4.049,7.111,9.760,2.021,6.083,9.760,2.601,4.533,4.606,5.452,5.708,37.987,2.071,3.608,1.848,3.292,3.179,4.644,3.482,4.185,5.708,6.083,2.926,3.432,3.089,5.393,2.863,2.309,2.775,2.884,2.823,3.206,1.159,2.653,9.058


In [134]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_scale_fe1, y_fe1_conso, test_size=0.25, random_state=42 ) # 25% des données dans le jeu de test

In [135]:
X_train.shape

(1083, 52)

In [136]:
X_test.shape

(361, 52)

In [137]:
y_train.shape

(1083,)

In [138]:
y_test.shape

(361,)

### 1.3 - Tests de modèles sans validation croisée

Il s'agit d'évaluer quelques modèles sans utiliser la validation croisée, en partant d'une baseline, pour aller vers des modèles plus élaborés.

L'hyperparamètre "alpha" sera fixe dans un premier temps (pas de validation croisée pour optimiser l'hyperparamètre).

On fera une boucle sur chaque modèle, et on stockera les scrores dans un tableau. Mais d'abord créons les fonctions qui seront utlisées dans la boucle :

**Baseline avec DummyRegressor**

On va utiliser la stratégie de la moyenne : prédit la moyenne des valeurs cibles d'entraînement. Créons la fonction qui prend les jeux d'entraînement et de tests et en entrée, et retourne les scores MSE, RMSE, R2, MAE:

In [141]:
def fit_dummyRegressor(X_train, y_train, X_test, y_test):

    start_time = timeit.default_timer()
    
    # Initialisation du DummyRegressor avec la stratégie 'mean'
    dummy_regressor = DummyRegressor(strategy='mean')
    
    # Entraînement du modèle
    dummy_regressor.fit(X_train, y_train)
    
    # Prédiction sur les données de test
    y_pred = dummy_regressor.predict(X_test)

    elapsed = round(timeit.default_timer() - start_time, 2)
    
    mse = round(mean_squared_error(y_test, y_pred), 2)       # Erreur quadratique moyenne
    rmse = round(np.sqrt(mse), 2)                            # Racine carrée de l'erreur quadratique moyenne (RMSE)
    mae = round(mean_absolute_error(y_test, y_pred), 2)      # Erreur absolue moyenne
    r2 = round(r2_score(y_test, y_pred), 2)                  # Coefficient de détermination

    return mse, rmse, r2, mae, elapsed

**Modèle de régression Ridge**

La régression ridge nous permet de réduire l'amplitude des coefficients d'une régression linéaire et d'éviter le sur-apprentissage. On optimisera l'hyperparamètre lors de la validation croisée avec GridSearchCV.

Créons une fonction qui permet d'instancier le modèle, l'entraîner, et calculer les scores:

In [143]:
def fit_ridge(X_train, y_train, X_test, y_test, alpha):

    start_time = timeit.default_timer()
    
    # Initialisation du modèle Ridge avec un paramètre alpha
    ridge_regressor = Ridge(alpha=alpha)  # alpha contrôle la régularisation ; plus grand, plus de régularisation
    
    # Entraînement du modèle
    ridge_regressor.fit(X_train, y_train)
    
    # Prédiction sur les données de test
    y_pred = ridge_regressor.predict(X_test)

    elapsed = round(timeit.default_timer() - start_time, 2)
    
    # Évaluation du modèle avec différentes métriques
    mse = round(mean_squared_error(y_test, y_pred), 2)       # Erreur quadratique moyenne
    rmse = round(np.sqrt(mse), 2)                            # Racine carrée de l'erreur quadratique moyenne (RMSE)
    mae = round(mean_absolute_error(y_test, y_pred), 2)      # Erreur absolue moyenne
    r2 = round(r2_score(y_test, y_pred), 2)                  # Coefficient de détermination

    return mse, rmse, r2, mae, elapsed

**Modèle de régression Lasso**

Le Lasso est une méthode de sélection de variables et de réduction de dimension supervisée : les variables qui ne sont pas nécessaires à la prédiction de l'étiquette sont éliminées.

Créons une fonction qui permet d'instancier le modèle, l'entraîner, et calculer les scores:

In [145]:
def fit_lasso(X_train, y_train, X_test, y_test, alpha):

    start_time = timeit.default_timer()
    
    # Initialisation du modèle Lasso avec un paramètre alpha
    lasso_regressor = Lasso(alpha=alpha)  # alpha contrôle la régularisation ; plus grand, plus de régularisation
    
    # Entraînement du modèle
    lasso_regressor.fit(X_train, y_train)
    
    # Prédiction sur les données de test
    y_pred = lasso_regressor.predict(X_test)

    elapsed = round(timeit.default_timer() - start_time, 2)
    
    # Évaluation du modèle avec différentes métriques
    mse = round(mean_squared_error(y_test, y_pred), 2)       # Erreur quadratique moyenne
    rmse = round(np.sqrt(mse), 2)                            # Racine carrée de l'erreur quadratique moyenne (RMSE)
    mae = round(mean_absolute_error(y_test, y_pred), 2)      # Erreur absolue moyenne
    r2 = round(r2_score(y_test, y_pred), 2)                  # Coefficient de détermination

    return mse, rmse, r2, mae, elapsed

**Modèle ElasticNet**

La méthode elastic net qui combine les deux termes de régularisation en un (Ridge L2 et Lasso L1).

Créons une fonction qui permet d'instancier le modèle, l'entraîner, et calculer les scores:

In [147]:
def fit_elasticNet(X_train, y_train, X_test, y_test, alpha):

    start_time = timeit.default_timer()

    # Initialisation du modèle ElasticNet
    elastic_net = ElasticNet(
        alpha=alpha, 
        l1_ratio=0.5, # Mélange égal entre ridge et lasso
        max_iter=1000, random_state=42
    )

    # Entraînement du modèle
    elastic_net.fit(X_train, y_train)

    # Prédictions sur le jeu de test
    y_pred = elastic_net.predict(X_test)
    
    elapsed = round(timeit.default_timer() - start_time, 2)
    
    # Évaluation du modèle avec différentes métriques
    mse = round(mean_squared_error(y_test, y_pred), 2)       # Erreur quadratique moyenne
    rmse = round(np.sqrt(mse), 2)                            # Racine carrée de l'erreur quadratique moyenne (RMSE)
    mae = round(mean_absolute_error(y_test, y_pred), 2)      # Erreur absolue moyenne
    r2 = round(r2_score(y_test, y_pred), 2)                  # Coefficient de détermination

    return mse, rmse, r2, mae, elapsed

**Bagging - Modèle RamdonForestRegressor**

Appelé aussi le bagging qui, appliqué aux arbres de décision, donne naissance au modèle de forêt aléatoire. 

Une forêt aléatoire est un ensemble de nombreux arbres de décision qui sont combinés pour produire une prédiction plus précise et plus robuste. Chaque arbre de décision est construit à partir d'un échantillon aléatoire des données et les résultats sont moyennés pour obtenir la prédiction finale. 

Le modèle de forêt aléatoire est intrinsèquement parallèle. Les arbres sont entraînés en même temps sur des parties du dataset.

Créons une fonction qui permet d'instancier le modèle, l'entraîner, et calculer les scores:

In [149]:
def fit_ramdomForestRegressor(X_train, y_train, X_test, y_test, X_fe1):

    start_time = timeit.default_timer()
    
    # Création du modèle
    # n_estimators : Nombre d'arbres dans la forêt. Valeur par défaut = 100. Une valeur plus élevée peut améliorer la précision mais augmente le temps de calcul.
    model = RandomForestRegressor(n_estimators=100, random_state=42)  # random_state permet
    
    # Entraînement du modèle
    model.fit(X_train, y_train)
    
    # Prédictions
    y_pred = model.predict(X_test)

    elapsed = round(timeit.default_timer() - start_time, 2)
    
    # Évaluation du modèle avec différentes métriques
    mse = round(mean_squared_error(y_test, y_pred), 2)       # Erreur quadratique moyenne
    rmse = round(np.sqrt(mse), 2)                            # Racine carrée de l'erreur quadratique moyenne (RMSE)
    mae = round(mean_absolute_error(y_test, y_pred), 2)      # Erreur absolue moyenne
    r2 = round(r2_score(y_test, y_pred), 2)                  # Coefficient de détermination

    # Afficher l'importance des features
    print("Importance des features dans le RandomForestRegressor :")
    importances = model.feature_importances_
    # Création d'un DataFrame pour afficher l'importance des features
    feature_importance = pd.DataFrame({'Feature': X_fe1.columns, 'Importance': importances})
    # Tri par ordre d'importance décroissante
    feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

    # Afficher les résultats
    print(feature_importance)

    return mse, rmse, r2, mae, elapsed

**Boosting - Modèle GradientBoostingRegressor**

Le boosting enchaîne l'entraînement des prédicteurs faibles de façon séquentielle, en se concentrant à chaque itération sur les échantillons qui ont généré le plus d'erreurs.

Il s'agit d'une méthode d'ensemble qui construit un modèle prédictif puissant en combinant plusieurs modèles faibles (typiquement des arbres de décision) de manière séquentielle

Le modèle de forêt aléatoire est intrinsèquement parallèle. Les arbres sont entraînés en même temps sur des parties du dataset.

Créons une fonction qui permet d'instancier le modèle, l'entraîner, et calculer les scores:

In [151]:
def fit_gradientBoostingRegressor(X_train, y_train, X_test, y_test, X_fe1):

    start_time = timeit.default_timer()

    # Création du modèle de régression Gradient Boosting
    gb_regressor = GradientBoostingRegressor(
        n_estimators=100,  # Nombre d'arbres
        learning_rate=0.1,  # Taux d'apprentissage (ici valeur par défaut)
        max_depth=3,  # Profondeur maximale des arbres (ici valeur par défaut)
        random_state=42  # Pour la reproductibilité
    )

    # Entraînement du modèle
    gb_regressor.fit(X_train, y_train)

    # Prédictions sur le jeu de test
    y_pred = gb_regressor.predict(X_test)

    elapsed = round(timeit.default_timer() - start_time, 2)
    
    # Évaluation du modèle avec différentes métriques
    mse = round(mean_squared_error(y_test, y_pred), 2)       # Erreur quadratique moyenne
    rmse = round(np.sqrt(mse), 2)                            # Racine carrée de l'erreur quadratique moyenne (RMSE)
    mae = round(mean_absolute_error(y_test, y_pred), 2)      # Erreur absolue moyenne
    r2 = round(r2_score(y_test, y_pred), 2)                  # Coefficient de détermination

    return mse, rmse, r2, mae, elapsed

**Modèle SVR**

Pour effectuer une régression avec un SVM (Support Vector Machine), on utilise le modèle appelé Support Vector Regression (SVR), qui fait partie des algorithmes de régression basés sur les SVM.

Le SVR cherche à trouver une fonction qui ne s'écarte pas trop des valeurs cibles, avec un contrôle sur la marge d'erreur autorisée.

Créons une fonction qui permet d'instancier le modèle, l'entraîner, et calculer les scores:

In [153]:
def fit_SVR(X_train, y_train, X_test, y_test):
    
    start_time = timeit.default_timer()
    
    # Initialisation du modèle SVR
    svr_model = SVR(
        kernel='rbf', 
        C=1.0,  # Paramètre de régularisation qui contrôle la pénalité pour les erreurs. Un C élevé cherche à minimiser les erreurs.
        epsilon=0.1 # Contrôle la largeur de la marge autour de la fonction cible. Les points situés dans cette marge ne contribuent pas à la fonction de coût
    )

    # Entraînement du modèle SVR
    svr_model.fit(X_train, y_train)

    # Prédictions sur le jeu de test
    y_pred = svr_model.predict(X_test)

    elapsed = round(timeit.default_timer() - start_time, 2)
    
    # Évaluation du modèle avec différentes métriques
    mse = round(mean_squared_error(y_test, y_pred), 2)       # Erreur quadratique moyenne
    rmse = round(np.sqrt(mse), 2)                            # Racine carrée de l'erreur quadratique moyenne (RMSE)
    mae = round(mean_absolute_error(y_test, y_pred), 2)      # Erreur absolue moyenne
    r2 = round(r2_score(y_test, y_pred), 2)                  # Coefficient de détermination

    return mse, rmse, r2, mae, elapsed

**1ere itération des modèles pour différentes valeurs d'alpha**

Créons une fonction qui permet de :
- lancer tous les modèles précédentsn avec plusieurs valeurs d'alpha pour Ridge, Lasso et ElasticNet
- afficher les scores

In [155]:
warnings.filterwarnings("ignore")

mse, rmse, r2, mae, elapsed = fit_dummyRegressor(X_train, y_train, X_test, y_test)
scores_array = np.array([['DummyRegressor', '', mse, rmse, r2, mae, elapsed]])

alphas = np.logspace(-6, 6, 7)

for alpha in alphas:
    mse, rmse, r2, mae, elapsed = fit_ridge(X_train, y_train, X_test, y_test, alpha)
    scores_array = np.vstack([scores_array, ['ridge', alpha, mse, rmse, r2, mae, elapsed]])
    mse, rmse, r2, mae, elapsed = fit_lasso(X_train, y_train, X_test, y_test, alpha)
    scores_array = np.vstack([scores_array, ['lasso', alpha, mse, rmse, r2, mae, elapsed]])
    mse, rmse, r2, mae, elapsed = fit_elasticNet(X_train, y_train, X_test, y_test, alpha)
    scores_array = np.vstack([scores_array, ['elasticNet', alpha, mse, rmse, r2, mae, elapsed]])

mse, rmse, r2, mae, elapsed = fit_ramdomForestRegressor(X_train, y_train, X_test, y_test, X_fe1)
scores_array = np.vstack([scores_array, ['RamdomForestRegressor', '', mse, rmse, r2, mae, elapsed]])

mse, rmse, r2, mae, elapsed = fit_gradientBoostingRegressor(X_train, y_train, X_test, y_test, X_fe1)
scores_array = np.vstack([scores_array, ['gradientBoostingRegressor', '', mse, rmse, r2, mae, elapsed]])

mse, rmse, r2, mae, elapsed = fit_SVR(X_train, y_train, X_test, y_test)
scores_array = np.vstack([scores_array, ['SVR', '', mse, rmse, r2, mae, elapsed]])

Importance des features dans le RandomForestRegressor :
                                            Feature  Importance
2                                  PropertyGFATotal       0.505
21  PrimaryPropertyType_Supermarket / Grocery Store       0.088
49                              electricity_percent       0.062
1                                    NumberofFloors       0.049
50                                      gaz_percent       0.040
23                    PrimaryPropertyType_Warehouse       0.033
13                        PrimaryPropertyType_Other       0.025
7                    PrimaryPropertyType_Laboratory       0.018
16                   PrimaryPropertyType_Restaurant       0.013
19        PrimaryPropertyType_Senior Care Community       0.012
47                   YearBuilt_Bin_(1992.0, 2003.5]       0.010
45                   YearBuilt_Bin_(1969.0, 1980.5]       0.010
29                            Neighborhood_DOWNTOWN       0.009
11           PrimaryPropertyType_Mixed Use Prope

In [156]:
# Conversion de l'array en DataFrame
df_scores_fe1 = pd.DataFrame(scores_array, columns=['Modèle', 'Alpha', 'MSE', 'RMSE', 'R2', 'MAE', 'ELAPSED_TIME'])

# on transforme la colonne R2 en numérique
df_scores_fe1['R2'] = pd.to_numeric(df_scores_fe1['R2'], errors='coerce')

# On trie le dataframe sur la colonne R2 du pmus grand au plus petit
df_scores_fe1.sort_values(by='R2', ascending=False, inplace=True)
df_scores_fe1.reset_index(inplace=True)

df_scores_fe1.head(30)

,index,Modèle,Alpha,MSE,RMSE,R2,MAE,ELAPSED_TIME
0,11,lasso,1.0,4882121725575.46,2209552.38,0.650,1525093.94,0.04
1,17,lasso,10000.0,4856026422771.06,2203639.36,0.650,1517734.68,0.0
2,2,lasso,1e-06,4882121086029.55,2209552.24,0.650,1525092.93,0.04
3,3,elasticNet,1e-06,4882122400011.25,2209552.53,0.650,1525093.81,0.04
4,4,ridge,0.0001,4884068398527.8,2209992.85,0.650,1525823.73,0.0
5,5,lasso,0.0001,4882121086092.74,2209552.24,0.650,1525092.93,0.04
6,6,elasticNet,0.0001,4882244868261.05,2209580.25,0.650,1525176.52,0.04
7,7,ridge,0.01,4883960654827.09,2209968.47,0.650,1525821.56,0.0
8,8,lasso,0.01,4882121092412.73,2209552.24,0.650,1525092.94,0.04
9,9,elasticNet,0.01,4879886330341.66,2209046.48,0.650,1527430.12,0.04


Le tableau ci-dessus est trié par R2. Plus le score R2 est proche de 1 mieux sait.

On observe :
- Sans surprise, le  modèle DummyRegressor a des mauvais scores, comme les modèles Ridge et Lasso avec alpha très élevé.
- Le Lasso est le meilleur modèle pour le moment. Mais très proche du modèle ElasticNet qui combine Lasso et Ridge. Il serait intéressant de voir ces scores avec une approche de validation croisée et d'optimisation des hyperparamètres.
- Pour le RandomForestRegressor, la variable PropertyGFATotal à la plus grande importance, et de loin. Il sera intéressant dans le 2ème feature engineering d'utiliser également la surface de stationnement qui pourrait également avoir une grande importance.
- Le RandomForestRegressor est le modèle le plus lent, suivi par le gradientBoostingRegressor. Une optimlisation des hyperparamètres permettra peut-être de réduire leur temps de traitement.

### 1.4 - Validation croisée avec le modèle Lasso

L'utilisation de GridSearchCV permet d'optimiser les hyperparamètres du modèle, en particulier le paramètre de régularisation alpha. 

GridSearchCV effectue une recherche exhaustive sur un ensemble de valeurs d'hyperparamètres spécifiés, en combinant ces valeurs avec la validation croisée pour évaluer chaque combinaison. Cela permet de trouver les meilleurs hyperparamètres pour le modèle.

In [159]:
# Définition des hyperparamètres à tester
param_grid = {
    #'alpha': [0.01, 0.1, 1.0, 10.0],  # Différentes valeurs de régularisation
    'alpha': np.logspace(-6, 6, 13) 
}

In [160]:
pd.set_option('display.float_format', '{:.3f}'.format)  # Désactiver l'écriture scientifique

Créons une fonction pour calculer le RMSE qui n'a pas directement disponible dans le GridSearchCV : 

In [162]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [163]:
rmse_scorer = make_scorer(rmse, greater_is_better=False)  # False car on minimise le RMSE

Choisissons les scores :

In [165]:
# Définition du dictionnaire des métriques de scoring
scoring = {
    'MAE': 'neg_mean_absolute_error',  # Utilise l'erreur absolue moyenne
    'R2': 'r2',                        # Utilise le coefficient de détermination
    'RMSE': rmse_scorer                # Utilise Root Mean Squared Error (racine carré de l'erreur quadratique moyenne)
}

Initialisons et entrainons le GridSearchCV :

In [167]:
# Initialisation de GridSearchCV
grid_search = GridSearchCV(
    estimator=Lasso(),           # une régression Lasso
    param_grid=param_grid,
    cv=5,
    scoring=scoring,
    refit='R2',  # Refit avec la meilleure valeur de R²
    #n_jobs=-1,         # Utilisation de tous les cœurs disponibles
    verbose=1
)

# Entraînement de GridSearchCV
grid_search.fit(X_train, y_train)



Fitting 5 folds for each of 13 candidates, totalling 65 fits


GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01,
       1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06])},
             refit='R2',
             scoring={'MAE': 'neg_mean_absolute_error', 'R2': 'r2',
                      'RMSE': make_scorer(rmse, greater_is_better=False)},
             verbose=1)

Affichons les résultats :

In [169]:
# Afficher les meilleurs paramètres trouvés
print(f"Meilleurs paramètres : {grid_search.best_params_}")

# Afficher le meilleur score
print("Meilleu(s) score sur le jeu d'entraînement:")
print(grid_search.best_score_)

# Utiliser le modèle avec les meilleurs paramètres
best_model = grid_search.best_estimator_

# Afficher les performances correspondantes
print("Résultats de la validation croisée :")
for score_name in scoring.keys():
    
    print(f"\nScores pour '{score_name}':")    
    for mean, std, params, mean_fit_time in zip(
            grid_search.cv_results_[f'mean_test_{score_name}'],  # score moyen pour chaque score
            grid_search.cv_results_[f'std_test_{score_name}'],   # écart-type du score
            grid_search.cv_results_['params'],                   # valeur de l'hyperparamètre
            grid_search.cv_results_['mean_fit_time']             # temps moyen d'entraînement
    ):
        print(f"{score_name} = {mean:.3f} (+/-{std * 2:.03f}) for {params}")

Meilleurs paramètres : {'alpha': 10000.0}
Meilleu(s) score sur le jeu d'entraînement:
0.5904473335057308
Résultats de la validation croisée :

Scores pour 'MAE':
MAE = -1542554.704 (+/-189210.398) for {'alpha': 1e-06}
MAE = -1542554.704 (+/-189210.398) for {'alpha': 1e-05}
MAE = -1542554.704 (+/-189210.399) for {'alpha': 0.0001}
MAE = -1542554.705 (+/-189210.403) for {'alpha': 0.001}
MAE = -1542554.709 (+/-189210.443) for {'alpha': 0.01}
MAE = -1542554.755 (+/-189210.850) for {'alpha': 0.1}
MAE = -1542555.208 (+/-189214.913) for {'alpha': 1.0}
MAE = -1542560.476 (+/-189256.227) for {'alpha': 10.0}
MAE = -1542596.487 (+/-189688.543) for {'alpha': 100.0}
MAE = -1542196.483 (+/-190822.699) for {'alpha': 1000.0}
MAE = -1535455.447 (+/-195365.197) for {'alpha': 10000.0}
MAE = -1536052.141 (+/-216132.916) for {'alpha': 100000.0}
MAE = -2176110.008 (+/-279706.375) for {'alpha': 1000000.0}

Scores pour 'R2':
R2 = 0.589 (+/-0.115) for {'alpha': 1e-06}
R2 = 0.589 (+/-0.115) for {'alpha': 1e-05}


Sous-forme de dataframe c'est mieux : 

In [171]:
# Liste pour stocker les résultats
results_fe1 = []

# Afficher les performances correspondantes
print("Résultats de la validation croisée :")
for score_name in scoring.keys():
       
    for mean, std, params, mean_fit_time in zip(
            grid_search.cv_results_[f'mean_test_{score_name}'],  # score moyen pour chaque score
            grid_search.cv_results_[f'std_test_{score_name}'],   # écart-type du score
            grid_search.cv_results_['params'],                   # valeur de l'hyperparamètre
            grid_search.cv_results_['mean_fit_time']             # temps moyen d'entraînement
    ):
                
        # Ajouter chaque combinaison de résultats à une liste sous forme de dictionnaire
        results_fe1.append({
            "score_name": score_name,
            "mean_score": mean,
            "std_score": std,
            "params": params,
            "mean_fit_time": mean_fit_time
        })

# Transformer en DataFrame
df_results_fe1 = pd.DataFrame(results_fe1)

# Convertir la colonne 'params' en chaîne de caractères
df_results_fe1['params'] = df_results_fe1['params'].apply(str)

# Transformation avec pivot
df_results_fe1 = df_results_fe1.pivot(
    index='params',                             # Les paramètres deviennent l'index
    columns='score_name',                       # Les valeurs uniques de score_name deviennent des colonnes
    values=['mean_score', 'mean_fit_time']      # Les valeurs à remplir dans les colonnes (ici, mean_score)
).reset_index()

# Aplatir les colonnes multi-indexées
df_results_fe1.columns = ['_'.join(col).strip() for col in df_results_fe1.columns.values]

# Réinitialiser l'index pour obtenir un DataFrame "normal"
df_results_fe1 = df_results_fe1.reset_index()
df_results_fe1.drop(columns=['index'], inplace=True)

# Supprimer l'axe des index
df_results_fe1 = df_results_fe1.rename_axis(None, axis=1)

# On trie le dataframe sur la colonne R2 du pmus grand au plus petit
df_results_fe1.sort_values(by='mean_score_R2', ascending=False, inplace=True)
df_results_fe1 = df_results_fe1.reset_index()
df_results_fe1.drop(columns=['index', 'mean_fit_time_MAE', 'mean_fit_time_RMSE'], inplace=True)
df_results_fe1.rename(columns={'mean_fit_time_R2': 'mean_fit_time'}, inplace=True)

df_results_fe1.head(30)

Résultats de la validation croisée :


,params_,mean_score_MAE,mean_score_R2,mean_score_RMSE,mean_fit_time
0,{'alpha': 10000.0},-1535455.447,0.590,-2252862.327,0.003
1,{'alpha': 1000.0},-1542196.483,0.589,-2256989.725,0.020
2,{'alpha': 1e-06},-1542554.704,0.589,-2257262.565,0.037
3,{'alpha': 1e-05},-1542554.704,0.589,-2257262.565,0.030
4,{'alpha': 0.0001},-1542554.704,0.589,-2257262.565,0.037
5,{'alpha': 0.001},-1542554.705,0.589,-2257262.565,0.030
6,{'alpha': 0.01},-1542554.709,0.589,-2257262.566,0.025
7,{'alpha': 0.1},-1542554.755,0.589,-2257262.571,0.033
8,{'alpha': 1.0},-1542555.208,0.589,-2257262.619,0.025
9,{'alpha': 10.0},-1542560.476,0.589,-2257262.656,0.031


Regardons les scores du modèle sur le jeu de test :

In [173]:
# Prédictions avec le modèle optimisé
y_pred = best_model.predict(X_test)

# Évaluation du modèle avec différentes métriques
mse = round(mean_squared_error(y_test, y_pred), 2)       # Erreur quadratique moyenne
rmse = round(np.sqrt(mse), 2)                            # Racine carrée de l'erreur quadratique moyenne (RMSE)
mae = round(mean_absolute_error(y_test, y_pred), 2)      # Erreur absolue moyenne
r2 = round(r2_score(y_test, y_pred), 2)                  # Coefficient de détermination

scores_cv_fe1 = np.array([['Lasso', mse, rmse, r2, mae]])

# Conversion de l'array en DataFrame
df_scores_cv_fe1 = pd.DataFrame(scores_cv_fe1, columns=['Modèle', 'MSE', 'RMSE', 'R2', 'MAE'])

# on transforme la colonne R2 en numérique
df_scores_cv_fe1['R2'] = pd.to_numeric(df_scores_cv_fe1['R2'], errors='coerce')

# On trie le dataframe sur la colonne R2 du pmus grand au plus petit
df_scores_cv_fe1.sort_values(by='R2', ascending=False, inplace=True)

df_scores_cv_fe1.head(30)

,Modèle,MSE,RMSE,R2,MAE
0,Lasso,4856026422771.06,2203639.36,0.650,1517734.68


Les scores sur le jeu de test sont meilleurs que ceux sur le jeu d'entraînement de la validation croisée.

## 2 - Amélioration du feature engineering  (cible = SiteEnergyUseWN(kBtu)) - 2ème feature engineering

In [492]:
# Charger le fichier de données
data_fe2 = pd.read_csv("C:/Users/admin/Documents/Projets/Projet_4/data_projet/cleaned/2016_Building_Energy_Benchmarking_fe2.csv", sep=',', low_memory=False)
data_fe2.head()

,NumberofBuildings,NumberofFloors,PropertyGFAParking,PropertyGFABuilding(s),SiteEnergyUseWN(kBtu),TotalGHGEmissions,Neighborhood_BALLARD,Neighborhood_CENTRAL,Neighborhood_DELRIDGE,Neighborhood_DELRIDGE NEIGHBORHOODS,Neighborhood_DOWNTOWN,Neighborhood_EAST,Neighborhood_GREATER DUWAMISH,Neighborhood_LAKE UNION,Neighborhood_MAGNOLIA / QUEEN ANNE,Neighborhood_NORTH,Neighborhood_NORTHEAST,Neighborhood_NORTHWEST,Neighborhood_SOUTHEAST,Neighborhood_SOUTHWEST,"YearBuilt_Bin_(1899.885, 1911.5]","YearBuilt_Bin_(1911.5, 1923.0]","YearBuilt_Bin_(1923.0, 1934.5]","YearBuilt_Bin_(1934.5, 1946.0]","YearBuilt_Bin_(1946.0, 1957.5]","YearBuilt_Bin_(1957.5, 1969.0]","YearBuilt_Bin_(1969.0, 1980.5]","YearBuilt_Bin_(1980.5, 1992.0]","YearBuilt_Bin_(1992.0, 2003.5]","YearBuilt_Bin_(2003.5, 2015.0]",PrimaryPropertyType_Distribution Center,PrimaryPropertyType_Hospital,PrimaryPropertyType_Hotel,PrimaryPropertyType_K-12 School,PrimaryPropertyType_Laboratory,PrimaryPropertyType_Large Office,PrimaryPropertyType_Low-Rise Multifamily,PrimaryPropertyType_Medical Office,PrimaryPropertyType_Mixed Use Property,PrimaryPropertyType_Office,PrimaryPropertyType_Other,PrimaryPropertyType_Refrigerated Warehouse,PrimaryPropertyType_Residence Hall,PrimaryPropertyType_Restaurant,PrimaryPropertyType_Retail Store,PrimaryPropertyType_Self-Storage Facility,PrimaryPropertyType_Senior Care Community,PrimaryPropertyType_Small- and Mid-Sized Office,PrimaryPropertyType_Supermarket / Grocery Store,PrimaryPropertyType_University,PrimaryPropertyType_Warehouse,PrimaryPropertyType_Worship Facility,electricity_percent,gaz_percent,steam_percent,usage_Autres,usage_Bureaux & Espaces de travail,usage_Commerce & Retail,usage_Entrepôts et Logistique,usage_Hébergement & Logement,usage_Loisirs et Divertissement,usage_Restauration,usage_Services publics & Infrastructure,usage_Soins médicaux,usage_Transports & Parking,usage_Éducation
0,1.000,12,0,88434,7456910.000,249.980,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,54.610,17.660,27.730,0.000,0.000,0.000,0.000,100.000,0.000,0.000,0.000,0.000,0.000,0.000
1,1.000,11,15064,88502,8664479.000,295.860,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,38.660,61.340,0.000,0.000,0.000,0.000,0.000,80.990,0.000,4.460,0.000,0.000,14.550,0.000
2,1.000,10,0,61320,6946800.500,286.430,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,40.750,26.660,32.590,0.000,0.000,0.000,0.000,100.000,0.000,0.000,0.000,0.000,0.000,0.000
3,1.000,18,62000,113580,14656503.000,505.010,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,37.880,62.120,0.000,0.000,0.000,0.000,0.000,64.480,0.000,0.000,0.000,0.000,35.520,0.000
4,1.000,2,37198,60090,12581712.000,301.810,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,60.990,39.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,100.000,0.000,0.000,0.000


In [494]:
data_fe2.shape

(1440, 66)

In [408]:
data_fe2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440 entries, 0 to 1439
Data columns (total 44 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   NumberofBuildings                        1440 non-null   float64
 1   NumberofFloors                           1440 non-null   int64  
 2   PropertyGFAParking                       1440 non-null   int64  
 3   PropertyGFABuilding(s)                   1440 non-null   int64  
 4   SiteEnergyUseWN(kBtu)                    1440 non-null   float64
 5   TotalGHGEmissions                        1440 non-null   float64
 6   Neighborhood_BALLARD                     1440 non-null   float64
 7   Neighborhood_CENTRAL                     1440 non-null   float64
 8   Neighborhood_DELRIDGE                    1440 non-null   float64
 9   Neighborhood_DELRIDGE NEIGHBORHOODS      1440 non-null   float64
 10  Neighborhood_DOWNTOWN                    1440 no

### 2.1 - Sélectionner les features et la cible :

In [496]:
y_fe2_conso = data_fe2['SiteEnergyUseWN(kBtu)']
X_fe2 = data_fe2.drop('SiteEnergyUseWN(kBtu)', axis=1, inplace=False)
X_fe2.shape

y_fe2_emissions = data_fe2['TotalGHGEmissions']
X_fe2 = X_fe2.drop('TotalGHGEmissions', axis=1, inplace=False)
X_fe2.shape

(1440, 64)

### 2.2 - Standardiser les valeurs et créer les jeux d'entraînement / test

In [498]:
X_scale_fe2 = StandardScaler().fit_transform(X_fe2)

In [500]:
df_fe2 = pd.DataFrame(X_scale_fe2)
df_fe2.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63
count,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000
mean,0.000,0.000,0.000,0.000,0.000,-0.000,-0.000,0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,-0.000,-0.000,0.000,0.000,-0.000,-0.000,-0.000,-0.000,0.000,-0.000,-0.000,-0.000,0.000,0.000,0.000,0.000,-0.000,-0.000,0.000,0.000,-0.000,0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.000,0.000,-0.000,-0.000,0.000,-0.000,0.000,0.000,-0.000,0.000,-0.000,0.000
std,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
min,-0.132,-0.820,-0.288,-1.009,-0.216,-0.184,-0.175,-0.026,-0.481,-0.278,-0.542,-0.303,-0.315,-0.216,-0.288,-0.239,-0.175,-0.165,-0.341,-0.290,-0.324,-0.184,-0.350,-0.434,-0.361,-0.347,-0.355,-0.311,-0.192,-0.026,-0.209,-0.306,-0.053,-0.278,-0.037,-0.141,-0.269,-0.046,-0.415,-0.088,-0.119,-0.088,-0.247,-0.138,-0.103,-0.495,-0.165,-0.103,-0.385,-0.221,-2.599,-1.092,-0.202,-0.269,-0.761,-0.414,-0.523,-0.301,-0.159,-0.158,-0.296,-0.094,-0.367,-0.345
25%,-0.132,-0.553,-0.288,-0.602,-0.216,-0.184,-0.175,-0.026,-0.481,-0.278,-0.542,-0.303,-0.315,-0.216,-0.288,-0.239,-0.175,-0.165,-0.341,-0.290,-0.324,-0.184,-0.350,-0.434,-0.361,-0.347,-0.355,-0.311,-0.192,-0.026,-0.209,-0.306,-0.053,-0.278,-0.037,-0.141,-0.269,-0.046,-0.415,-0.088,-0.119,-0.088,-0.247,-0.138,-0.103,-0.495,-0.165,-0.103,-0.385,-0.221,-0.803,-1.092,-0.202,-0.269,-0.761,-0.414,-0.523,-0.301,-0.159,-0.158,-0.296,-0.094,-0.367,-0.345
50%,-0.132,-0.286,-0.288,-0.348,-0.216,-0.184,-0.175,-0.026,-0.481,-0.278,-0.542,-0.303,-0.315,-0.216,-0.288,-0.239,-0.175,-0.165,-0.341,-0.290,-0.324,-0.184,-0.350,-0.434,-0.361,-0.347,-0.355,-0.311,-0.192,-0.026,-0.209,-0.306,-0.053,-0.278,-0.037,-0.141,-0.269,-0.046,-0.415,-0.088,-0.119,-0.088,-0.247,-0.138,-0.103,-0.495,-0.165,-0.103,-0.385,-0.221,0.001,-0.084,-0.202,-0.269,-0.761,-0.414,-0.523,-0.301,-0.159,-0.158,-0.296,-0.094,-0.367,-0.345
75%,-0.132,0.247,-0.288,0.163,-0.216,-0.184,-0.175,-0.026,-0.481,-0.278,-0.542,-0.303,-0.315,-0.216,-0.288,-0.239,-0.175,-0.165,-0.341,-0.290,-0.324,-0.184,-0.350,-0.434,-0.361,-0.347,-0.355,-0.311,-0.192,-0.026,-0.209,-0.306,-0.053,-0.278,-0.037,-0.141,-0.269,-0.046,-0.415,-0.088,-0.119,-0.088,-0.247,-0.138,-0.103,-0.495,-0.165,-0.103,-0.385,-0.221,1.159,0.818,-0.202,-0.269,0.913,-0.414,-0.523,-0.301,-0.159,-0.158,-0.296,-0.094,-0.367,-0.345
max,12.515,25.594,12.040,11.246,4.637,5.444,5.700,37.934,2.077,3.603,1.845,3.302,3.174,4.637,3.477,4.179,5.700,6.074,2.933,3.443,3.084,5.444,2.859,2.306,2.770,2.879,2.819,3.215,5.219,37.934,4.796,3.272,18.947,3.603,26.814,7.101,3.721,21.886,2.407,11.398,8.426,11.398,4.043,7.234,9.747,2.022,6.074,9.747,2.597,4.527,1.159,2.652,9.046,4.509,1.834,3.037,2.199,3.859,8.574,10.992,3.805,11.920,6.989,2.988


In [502]:
X_fe2_train, X_fe2_test, y_fe2_train, y_fe2_test = model_selection.train_test_split(X_scale_fe2, y_fe2_conso, test_size=0.25, random_state=42 ) # 25% des données dans le jeu de test

### 2.3 - Validation croisée avec le modèle Lasso

L'utilisation de GridSearchCV permet d'optimiser les hyperparamètres du modèle, en particulier le paramètre de régularisation alpha. 

GridSearchCV effectue une recherche exhaustive sur un ensemble de valeurs d'hyperparamètres spécifiés, en combinant ces valeurs avec la validation croisée pour évaluer chaque combinaison. Cela permet de trouver les meilleurs hyperparamètres pour le modèle.

**Créons une fonction réutilisable pour le GridSearcCV avec Lasso:**

In [487]:
def fit_gridSearchCV_lasso(X_train, y_train):

    # Définition des hyperparamètres à tester
    param_grid = {
        'alpha': np.logspace(-6, 6, 13) 
    }

    #rmse_scorer = make_scorer(rmse, greater_is_better=False)  # False car on minimise le RMSE

    # Définition du dictionnaire des métriques de scoring
    scoring = {
        'MAE': 'neg_mean_absolute_error',  # Utilise l'erreur absolue moyenne
        'R2': 'r2',                        # Utilise le coefficient de détermination
        'RMSE': 'neg_root_mean_squared_error'                # Utilise Root Mean Squared Error (racine carré de l'erreur quadratique moyenne)
    }

    # Initialisation de GridSearchCV
    grid_search = GridSearchCV(
        estimator=Lasso(),           # une régression Lasso
        param_grid=param_grid,
        cv=5,
        scoring=scoring,
        refit='R2',  # Refit avec la meilleure valeur de R²
        verbose=1
    )

    # Entraînement de GridSearchCV
    grid_search.fit(X_train, y_train)

    # Afficher les meilleurs paramètres trouvés
    print(f"Meilleurs paramètres : {grid_search.best_params_}")
    
    # Afficher le meilleur score
    print("Meilleu(s) score sur le jeu d'entraînement:")
    print(grid_search.best_score_)
    
    # Utiliser le modèle avec les meilleurs paramètres
    best_model = grid_search.best_estimator_
    
    # Afficher les performances correspondantes
    print("Résultats de la validation croisée :")
    for score_name in scoring.keys():
        
        print(f"\nScores pour '{score_name}':")    
        for mean, std, params, mean_fit_time in zip(
                grid_search.cv_results_[f'mean_test_{score_name}'],  # score moyen pour chaque score
                grid_search.cv_results_[f'std_test_{score_name}'],   # écart-type du score
                grid_search.cv_results_['params'],                   # valeur de l'hyperparamètre
                grid_search.cv_results_['mean_fit_time']             # temps moyen d'entraînement
        ):
            print(f"{score_name} = {mean:.3f} (+/-{std * 2:.03f}) for {params}")
    
    return scoring, grid_search.cv_results_

**Créons une fonction d'afficher des résultats:**

In [457]:
def print_result_gridSearchCV(cv_results_):

    # Liste pour stocker les résultats
    results = []
    
    # Afficher les performances correspondantes
    print("Résultats de la validation croisée :")
    for score_name in scoring.keys():
           
        for mean, std, params, mean_fit_time in zip(
                cv_results_[f'mean_test_{score_name}'],  # score moyen pour chaque score
                cv_results_[f'std_test_{score_name}'],   # écart-type du score
                cv_results_['params'],                   # valeur de l'hyperparamètre
                cv_results_['mean_fit_time']             # temps moyen d'entraînement
        ):
                    
            # Ajouter chaque combinaison de résultats à une liste sous forme de dictionnaire
            results_fe1.append({
                "score_name": score_name,
                "mean_score": mean,
                "std_score": std,
                "params": params,
                "mean_fit_time": mean_fit_time
            })
    
    # Transformer en DataFrame
    df_results = pd.DataFrame(results)
    
    # Convertir la colonne 'params' en chaîne de caractères
    df_results['params'] = df_results['params'].apply(str)
    
    # Transformation avec pivot
    df_results = df_results.pivot(
        index='params',                             # Les paramètres deviennent l'index
        columns='score_name',                       # Les valeurs uniques de score_name deviennent des colonnes
        values=['mean_score', 'mean_fit_time']      # Les valeurs à remplir dans les colonnes (ici, mean_score)
    ).reset_index()
    
    # Aplatir les colonnes multi-indexées
    df_results.columns = ['_'.join(col).strip() for col in df_results.columns.values]
    
    # Réinitialiser l'index pour obtenir un DataFrame "normal"
    df_results = df_results_fe1.reset_index()
    df_results.drop(columns=['index'], inplace=True)
    
    # Supprimer l'axe des index
    df_results = df_results.rename_axis(None, axis=1)
    
    # On trie le dataframe sur la colonne R2 du pmus grand au plus petit
    df_results.sort_values(by='mean_score_R2', ascending=False, inplace=True)
    df_results = df_results.reset_index()
    df_results.drop(columns=['index', 'mean_fit_time_MAE', 'mean_fit_time_RMSE'], inplace=True)
    df_results.rename(columns={'mean_fit_time_R2': 'mean_fit_time'}, inplace=True)
    
    return df_results

In [504]:
scoring_fe2, cv_results_fe2 = fit_gridSearchCV_lasso(X_fe2_train, y_fe2_train)

Fitting 5 folds for each of 13 candidates, totalling 65 fits
Meilleurs paramètres : {'alpha': 10000.0}
Meilleu(s) score sur le jeu d'entraînement:
0.5955685539255559
Résultats de la validation croisée :

Scores pour 'MAE':
MAE = -1494687.307 (+/-210109.212) for {'alpha': 1e-06}
MAE = -1494687.308 (+/-210109.212) for {'alpha': 1e-05}
MAE = -1494687.308 (+/-210109.213) for {'alpha': 0.0001}
MAE = -1494687.310 (+/-210109.215) for {'alpha': 0.001}
MAE = -1494687.333 (+/-210109.236) for {'alpha': 0.01}
MAE = -1494687.559 (+/-210109.450) for {'alpha': 0.1}
MAE = -1494689.843 (+/-210111.623) for {'alpha': 1.0}
MAE = -1494709.468 (+/-210132.665) for {'alpha': 10.0}
MAE = -1494744.877 (+/-210303.420) for {'alpha': 100.0}
MAE = -1494091.231 (+/-211797.161) for {'alpha': 1000.0}
MAE = -1487639.774 (+/-217510.572) for {'alpha': 10000.0}
MAE = -1510593.957 (+/-162031.554) for {'alpha': 100000.0}
MAE = -2219348.965 (+/-262384.021) for {'alpha': 1000000.0}

Scores pour 'R2':
R2 = 0.592 (+/-0.127) for

In [449]:
df_results_fe2 = print_result_gridSearchCV(cv_results_fe2)
df_results_fe2.head(30)

Résultats de la validation croisée :


KeyError: 'params'

In [176]:
STOP

NameError: name 'STOP' is not defined

L'utilisation de GridSearchCV avec une régression Ridge permet d'optimiser les hyperparamètres du modèle, en particulier le paramètre de régularisation alpha. GridSearchCV effectue une recherche exhaustive sur un ensemble de valeurs d'hyperparamètres spécifiés, en combinant ces valeurs avec la validation croisée pour évaluer chaque combinaison. Cela permet de trouver les meilleurs hyperparamètres pour le modèle.

In [ ]:
# Choisir un score à optimiser, ici le MSE
scoring = {
    'MSE': make_scorer(mean_squared_error, greater_is_better=False),
    'R2': 'r2'
}

In [ ]:
# Définir une grille de paramètres à tester
param_grid = {
    'alpha': np.logspace(-6, 6, 13)
}

# Créer un classifieur kNN avec recherche d'hyperparamètre par validation croisée
grid_search = model_selection.GridSearchCV(
    estimator=Ridge(),           # une régression Ridge
    param_grid = param_grid,     # hyperparamètres à tester
    cv=5,                        # nombre de folds de validation croisée
    scoring=scoring,             # score à optimiser
    refit='R2'                  # score utilisé pour le choix de l'hyperparamètre
)
    
# Optimiser ce grid_search sur le jeu d'entraînement
grid_search.fit(X_train, y_train)
    
# Afficher le(s) hyperparamètre(s) optimaux
print("Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:")
print(grid_search.best_params_)

# Afficher le meilleur score
print("Meilleu(s) score sur le jeu d'entraînement:")
print(grid_search.best_score_)

# Afficher les performances correspondantes
print("Résultats de la validation croisée :")
for score_name in scoring.keys():
   print(f"\nScores pour '{score_name}':")
   for mean, std, params in zip(
            grid_search.cv_results_[f'mean_test_{score_name}'],  # score moyen pour chaque score
            grid_search.cv_results_[f'std_test_{score_name}'],   # écart-type du score
            grid_search.cv_results_['params']                    # valeur de l'hyperparamètre
    ):
       print(f"{score_name} = {mean:.3f} (+/-{std * 2:.03f}) for {params}")

Pour la valeur d'alpha = 1.0, le MSE est meilleur que le modèle de régression classique et la baseline avec Dummy.

Scores sur les prédictions avec les meilleurs hyperparamètres sur le fichier de test :

In [ ]:
y_pred = grid_search.predict(X_test)

# Calculer des métriques pour évaluer le modèle
rmse = mean_squared_error(y_test, y_pred, squared=False)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE sur le test: {rmse:.3f}")
print(f"MSE sur le test : {mse:.3f}")
print(f"R^2 sur le test : {r2:.3f}")

Si on regarde la valeur de R2, les prédictions sont meilleures sur le fichier de test.

### 1.5 - Modèle de régression Lasso en validation croisée

Le Lasso est une méthode de sélection de variables et de réduction de dimension supervisée : les variables qui ne sont pas nécessaires à la prédiction de l'étiquette sont éliminées.

On va utliser GridSearchCV avec l'estimateur Lasso()

In [ ]:
warnings.filterwarnings("ignore")

# Définir une grille de paramètres à tester
param_grid = {
    'alpha': np.logspace(-5, 1, 50)
}

# Créer un classifieur kNN avec recherche d'hyperparamètre par validation croisée
grid_search = model_selection.GridSearchCV(
    estimator=Lasso(),             # une régression Lasso
    param_grid = param_grid,     # hyperparamètres à tester
    cv=5,                        # nombre de folds de validation croisée
    scoring=scoring,             # score à optimiser
    refit='R2'                  # score utilisé pour le choix de l'hyperparamètre
)
    
# Optimiser ce grid_search sur le jeu d'entraînement
grid_search.fit(X_train, y_train)
    
# Afficher le(s) hyperparamètre(s) optimaux
print("Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:")
print(grid_search.best_params_)

# Afficher le meilleur score
print("Meilleu(s) score sur le jeu d'entraînement:")
print(grid_search.best_score_)

# Afficher les performances correspondantes
print("Résultats de la validation croisée :")
for score_name in scoring.keys():
   print(f"\nScores pour '{score_name}':")
   for mean, std, params in zip(
            grid_search.cv_results_[f'mean_test_{score_name}'],  # score moyen pour chaque score
            grid_search.cv_results_[f'std_test_{score_name}'],   # écart-type du score
            grid_search.cv_results_['params']                    # valeur de l'hyperparamètre
    ):
       print(f"{score_name} = {mean:.3f} (+/-{std * 2:.03f}) for {params}")

Le résultat du modèle Lasso est comparable à celui du modèle Ridge. 

Scores sur le fichier de test :

In [ ]:
y_pred = grid_search.predict(X_test)

# Calculer des métriques pour évaluer le modèle
rmse = mean_squared_error(y_test, y_pred, squared=False)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE sur le test: {rmse:.3f}")
print(f"MSE sur le test : {mse:.3f}")
print(f"R^2 sur le test : {r2:.3f}")

In [441]:
# Initialisation du modèle XGBoost pour régression
model = XGBRegressor(
    n_estimators=100,  # Nombre d'arbres
    random_state=42
)

# Entraînement du modèle
model.fit(X_train, y_train)

# Prédictions
y_pred = model.predict(X_test)

# Calculer des métriques pour évaluer le modèle
rmse = mean_squared_error(y_test, y_pred, squared=False)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE sur le test: {rmse:.3f}")
print(f"MSE sur le test : {mse:.3f}")
print(f"R^2 sur le test : {r2:.3f}")

RMSE sur le test: 2453020.661
MSE sur le test : 6017310362198.184
R^2 sur le test : 0.572


In [469]:
warnings.filterwarnings("ignore")

mse, rmse, r2, mae, elapsed = fit_dummyRegressor(X_fe2_train, y_fe2_train, X_fe2_test, y_fe2_test)
scores_array = np.array([['DummyRegressor', '', mse, rmse, r2, mae, elapsed]])

alphas = np.logspace(-6, 6, 7)

for alpha in alphas:
    mse, rmse, r2, mae, elapsed = fit_ridge(X_fe2_train, y_fe2_train, X_fe2_test, y_fe2_test, alpha)
    scores_array = np.vstack([scores_array, ['ridge', alpha, mse, rmse, r2, mae, elapsed]])
    mse, rmse, r2, mae, elapsed = fit_lasso(X_fe2_train, y_fe2_train, X_fe2_test, y_fe2_test, alpha)
    scores_array = np.vstack([scores_array, ['lasso', alpha, mse, rmse, r2, mae, elapsed]])
    mse, rmse, r2, mae, elapsed = fit_elasticNet(X_fe2_train, y_fe2_train, X_fe2_test, y_fe2_test, alpha)
    scores_array = np.vstack([scores_array, ['elasticNet', alpha, mse, rmse, r2, mae, elapsed]])

mse, rmse, r2, mae, elapsed = fit_ramdomForestRegressor(X_fe2_train, y_fe2_train, X_fe2_test, y_fe2_test, X_fe2)
scores_array = np.vstack([scores_array, ['RamdomForestRegressor', '', mse, rmse, r2, mae, elapsed]])

mse, rmse, r2, mae, elapsed = fit_gradientBoostingRegressor(X_fe2_train, y_fe2_train, X_fe2_test, y_fe2_test, X_fe2)
scores_array = np.vstack([scores_array, ['gradientBoostingRegressor', '', mse, rmse, r2, mae, elapsed]])

mse, rmse, r2, mae, elapsed = fit_SVR(X_fe2_train, y_fe2_train, X_fe2_test, y_fe2_test)
scores_array = np.vstack([scores_array, ['SVR', '', mse, rmse, r2, mae, elapsed]])

Importance des features dans le RandomForestRegressor :
                                    Feature  Importance
3                    PropertyGFABuilding(s)       0.481
34            usage_Entrepôts et Logistique       0.083
33                  usage_Commerce & Retail       0.046
28                      electricity_percent       0.045
29                              gaz_percent       0.038
1                            NumberofFloors       0.038
2                        PropertyGFAParking       0.033
39                     usage_Soins médicaux       0.027
37                       usage_Restauration       0.025
32       usage_Bureaux & Espaces de travail       0.020
31                             usage_Autres       0.015
36          usage_Loisirs et Divertissement       0.014
26           YearBuilt_Bin_(1992.0, 2003.5]       0.012
40               usage_Transports & Parking       0.011
41                          usage_Éducation       0.010
35             usage_Hébergement & Logement     

In [471]:
# Conversion de l'array en DataFrame
df_scores_fe2 = pd.DataFrame(scores_array, columns=['Modèle', 'Alpha', 'MSE', 'RMSE', 'R2', 'MAE', 'ELAPSED_TIME'])

# on transforme la colonne R2 en numérique
df_scores_fe2['R2'] = pd.to_numeric(df_scores_fe2['R2'], errors='coerce')

# On trie le dataframe sur la colonne R2 du pmus grand au plus petit
df_scores_fe2.sort_values(by='R2', ascending=False, inplace=True)
df_scores_fe2.reset_index(inplace=True)

df_scores_fe2.head(30)

,index,Modèle,Alpha,MSE,RMSE,R2,MAE,ELAPSED_TIME
0,23,gradientBoostingRegressor,,5417060384506.43,2327457.92,0.630,1562962.91,0.68
1,22,RamdomForestRegressor,,5789900952609.77,2406221.3,0.600,1607511.01,2.53
2,10,ridge,1.0,6384927491111.34,2526841.41,0.560,1774462.25,0.0
3,17,lasso,10000.0,6330459980798.22,2516040.54,0.560,1763405.89,0.0
4,2,lasso,1e-06,6385779045136.79,2527009.9,0.560,1774591.13,0.18
5,3,elasticNet,1e-06,6385778567892.55,2527009.81,0.560,1774591.07,0.13
6,4,ridge,0.0001,6385943549039.79,2527042.45,0.560,1774577.61,0.0
7,5,lasso,0.0001,6385779044545.68,2527009.9,0.560,1774591.13,0.07
8,6,elasticNet,0.0001,6385731458834.34,2527000.49,0.560,1774585.13,0.06
9,7,ridge,0.01,6385777550777.95,2527009.61,0.560,1774589.93,0.0


In [ ]:
FAIRE UNE BOUCLE SUR LES MODELES, ET AJOUTER LES AUTRES MODELES : random forest, light gbm, xgboost, gradientboosting regressor, elastic net
AJOUTER LES AUTRES METRICS MAE ET RMSE.